<a href="https://colab.research.google.com/github/eder-castro/SistemaMultiAgentes/blob/main/SistemaMultiAgente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no Instagram, LinkedIn, Twitter (X) e Facebook. Eles costumam anunciar os eventos por lá.
*   **Assinar a newsletter da Alura:** Cadastre seu e-mail no site da Alura para receber novidades sobre cursos, eventos e imersões.
*   **Verificar a página de eventos da Alura:** Consulte periodicamente a seção de eventos no site da Alura para ver se há alguma Imersão IA com Google Gemini programada.

Esses são os melhores caminhos para você não perder a próxima edição!

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio.


In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [7]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [31]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca notícias sobre o tópico indicado",
        tools=[google_search],
        instruction='''
        Você é um agente de pesquisa. Sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 3 notícias relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim,
        podendo ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        '''
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [32]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em Instagram. Com base na lista de lançamentos
         mais recentes e relevantes vindas do buscador, você deve:
        Usar a ferramenta de busca do google (google_search) para criar um plano sobre quais são os pontos
        mais relevantes que poderíamos abordar em um post sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais informaçõe sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar
        esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post que será
        criado posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [33]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para Instagram.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [34]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para Instagram, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para Instagram."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [35]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################

if not topico:
  print("Você esqueceu de informar um tópico!")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- Resultado do agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- Resultado do agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- Resultado do agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Corinthians
Maravilha! Vamos então criar o post sobre novidades em Corinthians

--- Resultado do agente 1 (Buscador) ---



> Para fornecer as últimas notícias relevantes sobre o Corinthians, focarei em lançamentos e eventos significativos do último mês (aproximadamente desde 17 de abril de 2025 até hoje).
> 
> 
> Com base nas notícias recentes sobre o Corinthians, destaco os seguintes pontos relevantes:
> 
> 1.  **Lançamento da Nova Camisa 2 (2025):** Em 14 de maio de 2025, o Corinthians e a Nike lançaram o novo uniforme número 2 para a temporada, inspirado no título mundial de 2000. O time masculino estreou a camisa em jogo contra o Racing em 15 de maio, enquanto o time feminino a vestirá em 17 de maio.
> 
> 2.  **Superávit Financeiro:** O Corinthians registrou um superávit de R$ 12,1 milhões no primeiro bimestre de 2025, com a maior arrecadação vindo de direitos de transmissão e patrocínios. No entanto, o Conselho de Orientação do clube informou que os balancetes divulgados não passaram por auditoria externa.
> 
> 3.  **Contratações na Base:** O Corinthians continua ativo nas contratações para as categorias de base, registrando um grande número de aquisições na gestão de Augusto Melo.
> 
> 


------------------------------------------------

--- Resultado do agente 2 (Planejador) ---



> Okay, com base nos lançamentos e eventos recentes sobre o Corinthians, vou usar a ferramenta de busca do Google para aprofundar as informações e criar um plano de conteúdo para um post no Instagram sobre o tema mais relevante.
> 
> 
> Após analisar as informações e os resultados da pesquisa, considero que o tema mais relevante para um post no Instagram sobre o Corinthians, no momento, é o **Lançamento da Nova Camisa 2 (2025)**.
> 
> **Justificativa:**
> 
> *   **Apelo Visual e Atualidade:** O lançamento de uma nova camisa é um evento que gera grande interesse entre os torcedores, especialmente quando há um design diferenciado ou uma homenagem envolvida. A nova camisa 2 do Corinthians tem um apelo visual forte, remetendo ao título mundial de 2000, o que a torna ainda mais atraente.
> *   **Engajamento:** Posts sobre lançamentos de produtos costumam gerar bastante engajamento, com comentários, curtidas e compartilhamentos.
> *   **Informação e História:** O post pode combinar informações sobre o design da camisa, os detalhes que remetem ao título mundial de 2000 e a história por trás dessa conquista.
> 
> **Pontos Relevantes a serem Abordados no Post:**
> 
> 1.  **Design da Camisa:**
>     *   Cor predominante preta com detalhes em branco nas mangas e faixas laterais.
>     *   Escudo e logo da Nike centralizados no peito, remetendo à camisa de 2000.
>     *   Ausência da borda branca no escudo, um pedido antigo da torcida.
>     *   Gola careca com padrão listrado na nuca.
>     *   Frase "Todo Poderoso Timão" na parte interna, em referência ao canto da torcida.
>     *   Número 8 estilizado na fonte dos números, em homenagem ao capitão da final do mundial de 2000.
> 2.  **Homenagem ao Título Mundial de 2000:**
>     *   A importância do título mundial de 2000 na história do Corinthians.
>     *   O impacto da conquista na identidade do clube e na torcida.
>     *   O legado daquele time e a memória dos jogadores que fizeram história.
> 3.  **Estreia da Camisa:**
>     *   O time masculino estreou a camisa contra o Racing em 15 de maio de 2025.
>     *   O time feminino a vestirá em 17 de maio de 2025.
> 4.  **Disponibilidade e Preços:**
>     *   A camisa está disponível nas lojas Nike, ShopTimão e Centauro.
>     *   Preço da versão jogador: R$ 699,99.
>     *   Preço da versão torcedor: R$ 349,99.
> 
> **Plano de Conteúdo para o Post:**
> 
> *   **Título:** Nova Camisa 2 do Corinthians: Uma Homenagem ao Mundial de 2000! 🖤
> *   **Imagem/Vídeo:**
>     *   Foto da nova camisa em detalhes (frente, costas, detalhes do escudo, gola, etc.).
>     *   Vídeo curto mostrando a camisa em movimento, com destaque para os detalhes.
>     *   Foto/vídeo da torcida vestindo a camisa.
>     *   Foto do time usando a camisa em jogo.
> *   **Texto:**
>     *   Começar com uma chamada atrativa sobre o lançamento da nova camisa.
>     *   Apresentar os detalhes do design e as referências ao título mundial de 2000.
>     *   Contar um pouco da história do título mundial e sua importância para o clube.
>     *   Informar sobre a estreia da camisa pelos times masculino e feminino.
>     *   Incluir informações sobre disponibilidade e preços.
>     *   Usar hashtags relevantes: #Corinthians #NovaCamisa #Timão #Mundial2000 #Nike #FielTorcida
> *   **Call to Action:**
>     *   Perguntar aos seguidores o que acharam da nova camisa.
>     *   Incentivar os torcedores a comprarem a camisa e mostrarem seu apoio ao time.
>     *   Convidar os seguidores a compartilharem suas memórias sobre o título mundial de 2000.
> 
> **Sugestões Adicionais:**
> 
> *   Criar um carrossel com várias fotos e vídeos da camisa.
> *   Fazer stories com enquetes e perguntas sobre a camisa e o título mundial.
> *   Marcar a Nike e o Corinthians nas publicações.
> *   Interagir com os comentários e mensagens dos seguidores.
> 
> Este plano de conteúdo visa criar um post informativo, visualmente atraente e que gere engajamento com a torcida corintiana, aproveitando o lançamento da nova camisa 2 e a forte ligação emocional com o título mundial de 2000.
> 


------------------------------------------------

--- Resultado do agente 3 (Redator) ---



> 🎉⚫️⚪️ SAIU A NOVA CAMISA 2 DO TIMÃO! ⚫️⚪️🎉
> 
> Diretamente do túnel do tempo para o futuro! 🚀 A nova camisa do Corinthians é uma homenagem ÉPICA ao título mundial de 2000! 🏆
> 
> 🖤 Design INOVADOR:
> ⚫️ Predominantemente preta, com detalhes em branco que dão um toque clássico.
> 🛡️ Escudo e logo da Nike centralizados, igual àquela camisa que nos deu o mundo!
> 🔥 Sem a borda branca no escudo: um pedido ATENDIDO da Fiel!
> ✨ Gola careca com padrão listrado que é puro estilo.
> 🤫 E tem mais: a frase "Todo Poderoso Timão" escondida na parte interna. Arrepiante!
> ⚽️ O número 8 estilizado é uma homenagem ao nosso eterno capitão da final de 2000.
> 
> 🗓️ ESTREIA:
> 👨‍👨‍👧‍👦 O time masculino já estreou contra o Racing em 15 de maio de 2025.
> 👩‍👩‍👧‍👦 O time feminino vestirá no dia 17 de maio de 2025.
> 
> 💰 PREÇOS E ONDE ENCONTRAR:
> 🛒 Disponível nas lojas Nike, ShopTimão e Centauro.
> 💸 Versão jogador: R$ 699,99
> 👕 Versão torcedor: R$ 349,99
> 
> E aí, Fiel? O que acharam dessa belezura? 😍 Corre pra garantir a sua e mostrar o orgulho de ser CORINTHIANS! 🖤⚪️
> 
> #Corinthians #NovaCamisa #VaiCorinthians #Mundial2000
> 


------------------------------------------------

--- Resultado do agente 4 (Revisor) ---



> O rascunho está quase perfeito, mas podemos fazer alguns ajustes para deixá-lo ainda mais engajador e informativo para a Fiel:
> 
> **Problemas e Sugestões:**
> 
> *   **Data da Estreia:** As datas de estreia (15 e 17 de maio de 2025) indicam que o post está situado no futuro. Isso pode causar confusão. Sugiro verificar se as datas estão corretas ou ajustar a redação para o tempo presente.
> *   **Tom:** O texto já está bom, mas podemos adicionar mais "Corinthianismo". Use mais gírias e expressões que a Fiel usa.
> *   **Chamada para Ação:** Podemos criar mais desejo na Fiel para que ela compre a camisa.
> 
> **Sugestão de Melhorias:**
> 
> 🎉⚫️⚪️ A NOVA ARMADURA DO TIMÃO CHEGOU! ⚫️⚪️🎉
> 
> Diretamente do túnel do tempo para o futuro! 🚀 A nova camisa do Corinthians é uma homenagem ÉPICA ao título mundial de 2000! Aquele título que fez a gente ZUAR TODO MUNDO! 🏆
> 
> 🖤 Design que FAZ CHORAR DE EMOÇÃO:
> ⚫️ Predominantemente preta, com detalhes em branco que dão um toque clássico.
> 🛡️ Escudo e logo da Nike centralizados, igual àquela camisa que nos deu o mundo!
> 🔥 Sem a borda branca no escudo: UM PEDIDO DA FIEL É UMA ORDEM!
> ✨ Gola careca com padrão listrado que é puro estilo.
> 🤫 E tem mais: a frase "Todo Poderoso Timão" escondida na parte interna. ARREPIANTE!
> ⚽️ O número 8 estilizado é uma homenagem ao nosso eterno capitão da final de 2000. ÍDOLO!
> 
> 🗓️ ESTREIA:
> 👨‍👨‍👧‍👦 O time masculino já ESTREOU contra o Racing.
> 👩‍👩‍👧‍👦 O time feminino também já ENTROU EM CAMPO com o novo manto.
> 
> 💰 PREÇOS E ONDE ENCONTRAR:
> 🛒 Disponível nas lojas Nike, ShopTimão e Centauro.
> 💸 Versão jogador: R$ 699,99 (Para você se sentir como um craque!)
> 👕 Versão torcedor: R$ 349,99 (Para mostrar sua paixão em qualquer lugar!)
> 
> E aí, Fiel? Babaram nessa belezura? 😍 Corre pra garantir a sua e mostrar o orgulho de ser CORINTHIANS! VAI CORINTHIANS! 🖤⚪️
> 
> #Corinthians #NovaCamisa #VaiCorinthians #Mundial2000 #Timão #FielTorcedor
> 


------------------------------------------------
